## **1. Importar Librerías**


In [ ]:
import pymssql
import os
import pandas as pd
from dotenv import load_dotenv

load_dotenv()


True

In [2]:
DB_SERVER = os.getenv("DB_SERVER")
DB_PORT = os.getenv("DB_PORT")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")

print(f"📡 Servidor: {DB_SERVER}")
print(f"📊 Base de datos: {DB_NAME}")


📡 Servidor: localhost
📊 Base de datos: ClimateDB


## **3. Conectar a la Base de Datos**


In [3]:
conn = pymssql.connect(
    server=DB_SERVER,
    port=DB_PORT,
    user=DB_USER,
    password=DB_PASSWORD,
    database=DB_NAME,
    autocommit=True
)

cursor = conn.cursor()
print("✅ Conectado a la base de datos ClimateDB")


✅ Conectado a la base de datos ClimateDB


## **4. Verificar Estructura Actual**


In [4]:
# Obtener columnas actuales de la tabla
cursor.execute("""
    SELECT COLUMN_NAME, DATA_TYPE, IS_NULLABLE
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME = 'climate_observations'
    ORDER BY ORDINAL_POSITION
""")

print("📋 Columnas actuales en climate_observations:")
print("-" * 50)
for row in cursor.fetchall():
    print(f"  • {row[0]:30} {row[1]:15} {row[2]}")


📋 Columnas actuales en climate_observations:
--------------------------------------------------
  • id                             bigint          NO
  • municipio                      nvarchar        NO
  • datetime                       datetime2       NO
  • wind_speed_10m                 float           YES
  • wind_direction_10m             int             YES
  • temperature_2m                 float           YES
  • relative_humidity_2m           int             YES
  • precipitation                  float           YES
  • created_at                     datetime2       YES


In [ ]:
# Verificar datos de ejemplo
cursor.execute("""
    SELECT TOP 3
        municipio,
        datetime,
        wind_speed_10m,
        temperature_2m
    FROM climate_observations
    ORDER BY datetime DESC
""")

print("\n📊 Datos de ejemplo (antes de la modificación):")
df_before = pd.DataFrame(
    cursor.fetchall(),
    columns=['municipio', 'datetime', 'wind_speed_10m', 'temperature_2m']
)
print(df_before)



📊 Datos de ejemplo (antes de la modificación):
   municipio            datetime  wind_speed_10m  temperature_2m
0  el_molino 2025-12-18 23:00:00             8.6            25.4
1    mingueo 2025-12-18 23:00:00             5.0            25.8
2     uribia 2025-12-18 23:00:00             9.4            26.7


## **5. Agregar Columnas Calculadas**

Vamos a agregar columnas calculadas persistentes que se mantienen automáticamente sincronizadas con la columna `datetime`.


In [6]:
# Verificar si las columnas ya existen
cursor.execute("""
    SELECT COUNT(*) FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME = 'climate_observations'
    AND COLUMN_NAME IN ('year', 'month', 'day', 'hour')
""")

existing_cols = cursor.fetchone()[0]

if existing_cols > 0:
    print(f"⚠️  Ya existen {existing_cols} columnas temporales.")
    print("   Se eliminarán y recrearán...\n")
    
    # Eliminar columnas existentes si las hay
    for col in ['year', 'month', 'day', 'hour']:
        try:
            cursor.execute(f"""
                IF EXISTS (
                    SELECT * FROM INFORMATION_SCHEMA.COLUMNS
                    WHERE TABLE_NAME = 'climate_observations'
                    AND COLUMN_NAME = '{col}'
                )
                ALTER TABLE climate_observations DROP COLUMN {col}
            """)
            print(f"   🗑️  Columna '{col}' eliminada")
        except Exception as e:
            print(f"   ℹ️  {col}: {str(e)[:50]}")

print("\n🔨 Creando columnas calculadas...")



🔨 Creando columnas calculadas...


In [ ]:
try:
    # Agregar columnas calculadas persistentes
    cursor.execute("""
        ALTER TABLE climate_observations
        ADD 
            year AS YEAR(datetime) PERSISTED,
            month AS MONTH(datetime) PERSISTED,
            day AS DAY(datetime) PERSISTED,
            hour AS DATEPART(HOUR, datetime) PERSISTED
    """)
    
    print("✅ Columnas agregadas exitosamente:")
    print("   • year   - Año extraído de datetime")
    print("   • month  - Mes extraído de datetime")
    print("   • day    - Día extraído de datetime")
    print("   • hour   - Hora extraída de datetime")
    
except Exception as e:
    print(f"❌ Error al crear columnas: {e}")


✅ Columnas agregadas exitosamente:
   • year   - Año extraído de datetime
   • month  - Mes extraído de datetime
   • day    - Día extraído de datetime
   • hour   - Hora extraída de datetime


## **6. Crear Índices para Optimización**

Los índices mejoran el rendimiento de las consultas que filtran por año, mes o combinaciones de estas columnas.


In [8]:
# Eliminar índices existentes si los hay
indexes_to_drop = [
    'IX_climate_observations_year_month',
    'IX_climate_observations_municipio_year_month',
    'IX_climate_observations_year'
]

for idx in indexes_to_drop:
    try:
        cursor.execute(f"""
            IF EXISTS (
                SELECT * FROM sys.indexes 
                WHERE name = '{idx}' 
                AND object_id = OBJECT_ID('climate_observations')
            )
            DROP INDEX {idx} ON climate_observations
        """)
    except:
        pass

print("🔍 Creando índices...")


🔍 Creando índices...


In [9]:
try:
    # Índice para consultas por año y mes
    cursor.execute("""
        CREATE INDEX IX_climate_observations_year_month
        ON climate_observations(year, month)
    """)
    print("✅ Índice IX_climate_observations_year_month creado")
    
except Exception as e:
    print(f"⚠️  IX_climate_observations_year_month: {str(e)[:80]}")


✅ Índice IX_climate_observations_year_month creado


In [10]:
try:
    # Índice para consultas por municipio, año y mes
    cursor.execute("""
        CREATE INDEX IX_climate_observations_municipio_year_month
        ON climate_observations(municipio, year, month)
    """)
    print("✅ Índice IX_climate_observations_municipio_year_month creado")
    
except Exception as e:
    print(f"⚠️  IX_climate_observations_municipio_year_month: {str(e)[:80]}")


✅ Índice IX_climate_observations_municipio_year_month creado


In [ ]:
try:
    # Índice simple por año
    cursor.execute("""
        CREATE INDEX IX_climate_observations_year
        ON climate_observations(year)
    """)
    print("✅ Índice IX_climate_observations_year creado")
    
except Exception as e:
    print(f"⚠️  IX_climate_observations_year: {str(e)[:80]}")


## **7. Verificar la Nueva Estructura**


In [11]:
# Verificar columnas después de la modificación
cursor.execute("""
    SELECT COLUMN_NAME, DATA_TYPE, IS_NULLABLE
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME = 'climate_observations'
    ORDER BY ORDINAL_POSITION
""")

print("📋 Columnas después de la modificación:")
print("-" * 50)
for row in cursor.fetchall():
    print(f"  • {row[0]:30} {row[1]:15} {row[2]}")


📋 Columnas después de la modificación:
--------------------------------------------------
  • id                             bigint          NO
  • municipio                      nvarchar        NO
  • datetime                       datetime2       NO
  • wind_speed_10m                 float           YES
  • wind_direction_10m             int             YES
  • temperature_2m                 float           YES
  • relative_humidity_2m           int             YES
  • precipitation                  float           YES
  • created_at                     datetime2       YES
  • year                           int             YES
  • month                          int             YES
  • day                            int             YES
  • hour                           int             YES


In [12]:
# Verificar índices creados
cursor.execute("""
    SELECT 
        i.name AS index_name,
        STRING_AGG(c.name, ', ') AS columns
    FROM sys.indexes i
    INNER JOIN sys.index_columns ic ON i.object_id = ic.object_id AND i.index_id = ic.index_id
    INNER JOIN sys.columns c ON ic.object_id = c.object_id AND ic.column_id = c.column_id
    WHERE i.object_id = OBJECT_ID('climate_observations')
    AND i.name IS NOT NULL
    GROUP BY i.name
    ORDER BY i.name
""")

print("\n🔍 Índices creados:")
print("-" * 70)
for row in cursor.fetchall():
    print(f"  • {row[0]:50} [{row[1]}]")



🔍 Índices creados:
----------------------------------------------------------------------
  • IX_climate_observations_municipio_year_month       [municipio, year, month]
  • IX_climate_observations_year_month                 [year, month]
  • PK__climate___3213E83FE7BE4ED9                     [id]
  • UQ_municipio_datetime                              [municipio, datetime]
